In [1]:
from fastai.vision.all import *
from fastbook import *

In [2]:
torch.cuda.is_available()

True

## Image loading

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
def loadImages(paths):
    images = [Image.open(p) for p in paths]
    classes = [p.parent.name for p in paths]
    return (images, classes)
    
loadImages([
    path/"training"/"5"/"0.png",
    path/"training"/"9"/"10003.png"
])

([<PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
  <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>],
 ['5', '9'])

In [5]:
import glob

def allImages(path):
    return list(path.glob("**/*.png"))

testImages = allImages(path)
assert len(testImages) > 0
for image in testImages: assert image.name.endswith(".png") 
len(testImages), type(testImages[0])

(70000, pathlib.WindowsPath)

## Model

In [20]:
class Model:
    def __init__(self, imageSize, categoryCount):
        self.w1 = self.__initParams(imageSize[0] * imageSize[1], categoryCount)
        self.b1 = self.__initParams(categoryCount)
        self.params = [self.w1, self.b1]
        
    def applyModel(self, batch):
        return batch@self.w1 + self.b1
    
    def fit(self, lr):
        for p in self.params:
            p.data -= p.grad.data * lr
            p.grad.zero_()
    
    def __initParams(self, *size):
        return torch.rand(size).requires_grad_()

## Learning preparation

In [7]:
def mse(targetProbabilities, predictedProbabilities):
    return torch.square(targetProbabilities - predictedProbabilities).mean()

testTargets = tensor([1, 0, 1])
test1 = mse(testTargets, tensor([0.9, 0.2, 0.4]))
test2 = mse(testTargets, tensor([0.9, 0.2, 0.5]))
test1, test2

(tensor(0.1367), tensor(0.1000))

In [8]:
def sigmoid(x): return 1/(1+torch.exp(-x))

sigmoid(tensor([-4, -1, 1, 4]))

tensor([0.0180, 0.2689, 0.7311, 0.9820])

In [24]:
def mse_loss(targetProbabilities, predictions):
    return mse(targetProbabilities, predictions.sigmoid())
    
test1 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 0]))
test2 = mse_loss(tensor([1, 0, 1]), tensor([10, -3, 1]))
test1, test2

(tensor(0.0841), tensor(0.0249))

In [10]:
def calculateAccuracy(targetPredictions, predictions):
    border = 0.5
    return ((predictions > border) == (targetPredictions > border)).float().mean()

calculateAccuracy(tensor([1, 0, 1]), tensor([0.2, 0, 0.61]))

tensor(0.6667)

In [11]:
class BatchLoader:
    def __init__(self, images, batchSize):
        self.images = images.copy()
        random.shuffle(self.images)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def nextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.images))
        batchEndIndex = min(startBatchFrom + self.batchSize, len(self.images))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            return self.images[batchStartIndex:batchEndIndex]

In [12]:
class BatchLoader:
    def __init__(self, items, batchSize):
        self.items = items.copy()
        random.shuffle(self.items)
        self.batchSize = batchSize
        self.nextBatch = 0 
    def getNextBatch(self):
        batchStartIndex = min(self.batchSize * self.nextBatch, len(self.items))
        batchEndIndex = min(batchStartIndex + self.batchSize, len(self.items))
        if batchStartIndex == batchEndIndex:
            return None
        else:
            self.nextBatch += 1
            return self.items[batchStartIndex:batchEndIndex]
       
    
testItems = [1, 2, 3, 4, 5, 6, 7]
testLoader = BatchLoader(testItems, 4)
receivedItems = testLoader.getNextBatch() + testLoader.getNextBatch()
receivedItems.sort()
assert testItems == receivedItems, "received aren't the same as tests " + receivedItems
assert testLoader.getNextBatch() == None

In [13]:
def loadTrainingBatch(paths):
    images = [tensor(Image.open(p)).view(-1) for p in paths]
    classes = [p.parent.name for p in paths]
    return (torch.stack(images).float()/255, classes)
    
testBatchLoader = BatchLoader(allImages(path/"training"), 100)
trainingImages, trainingClasses = loadTrainingBatch(testBatchLoader.getNextBatch())
assert len(trainingImages) == len(trainingClasses)

In [14]:
def numberToPrediction(number):
    return [1 if i == number else 0 for i in range(10)]

def targetPredictions(classes):
    return tensor([numberToPrediction(int(c)) for c in classes])

targetPredictions(["0", "5", "9"])

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

### The learning loop

In [25]:
def fitOneBatch(model, batch, lr):
    traingingImages, trainingClasses = loadTrainingBatch(batch)
    predictions = model.applyModel(traingingImages)
    target = targetPredictions(trainingClasses)
    loss = mse_loss(target, predictions)
    loss.backward()
    gradient = model.w1.grad
    #model.fit(lr)
    print("Loss: " + str(loss.item()))
    #print("Accuracy: " + str(accuracy))
    return gradient

grad = fitOneBatch(Model((28, 28), 10), allImages(path/"training/4")[:10], 10)

#hide_output
df = pd.DataFrame(grad)
# df.loc[df > 0.0]
df

Loss: 0.8999999761581421


,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
779,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def performEpoach(model, trainingSet, batchSize, lr):
    batchLoader = BatchLoader(trainingSet, batchSize)
    batch = batchLoader.getNextBatch()
    while batch != None:
        fitOneBatch(model, batch, lr)
        batch = batchLoader.getNextBatch()
    
model = Model((28, 28), 10)    
performEpoach(model, allImages(path/"training"), 1000, 2.)

Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
Loss: 0.8999999761581421
